In [ ]:
# Runtime의 Disk를 이용해서 tensorflow object detection API를 알아보아요

# 우리가 object detection model을 한땀한땀 구현하는건 상당히 어려운 일
# 이미 잘 만들어진  pretrained network을 이용해서 사용

# 우리가 사용할 모델은
# Tensorflow Model Garden이라는 repository에서 받아서 사용
# 이 repository는 최신의 model 그리고 예제 그리고 library 등을 제공하는
# tensorflow 공식 repository

In [25]:
# 1.코드처리를 하기위해 변수 2개만 먼저 설정하고 진행
# 하나는 환경변수, 다른 하나는 일반변를 만들 예정
import os

# 시스템 환경변수 설정
os.environ['ROOT_FOLDER'] = '/content/drive/MyDrive/data'

# 일반변수 생성
ROOT_FOLDER ='/content/drive/MyDrive/data'

In [26]:
# 변수가 잘 만들어졌는지 확인
print(ROOT_FOLDER)

# jupyter notebook에서 시스템명령어를 사용하려면 !이용
!echo "$ROOT_FOLDER"

/content/drive/MyDrive/data
/content/drive/MyDrive/data


In [27]:
# 2.ROOT_FOLDER아래에 특정 폴저를 만들어서 사용할 모든 파일을 해당 폴더 안에 저장해서 사용
!mkdir "$ROOT_FOLDER"/TF2_Object_Detection_API

mkdir: cannot create directory ‘/content/drive/MyDrive/data/TF2_Object_Detection_API’: File exists


In [28]:
# 3. working directory를 변경
# 현재의 working directory 를 확인하는 명령어  pwd
!pwd
# working directory 를 변경하려면 cd(change directory)명령을 이용
# cd는 느낌표 없이 사용 다른 주석이나 사용하지 않고 작성

/content/drive/MyDrive/data/TF2_Object_Detection_API/models/research


In [9]:
cd /content/drive/MyDrive/data/TF2_Object_Detection_API

/content/drive/MyDrive/data/TF2_Object_Detection_API


In [10]:
# 4. tensorflow model garden이라는 repository안에 있는 모델을 이용하고 일단 repository clone
!git clone https://github.com/tensorflow/models.git

fatal: destination path 'models' already exists and is not an empty directory.


In [29]:
# 5. 기본작업디렉토리(working directory)는
# models/research 폴더예요
# 해당 폴더로 working directory를 변경

In [30]:
cd models/research

[Errno 2] No such file or directory: 'models/research'
/content/drive/MyDrive/data/TF2_Object_Detection_API/models/research


In [31]:
# protocol buffer 필요
# colab에는 설치가 이미 되어잇어서 따로 설치할 필요가 없음
# windows에서 이 작업을 하려면 당연히 따로 protocol buffer를 설치


In [32]:
# 6. (.proto)파일을 찾아서 protoc를 이용해서 컴파일
# 위치는
!protoc object_detection/protos/*.proto --python_out=.

In [33]:
! ls object_detection/protos

anchor_generator_pb2.py		       keypoint_box_coder_pb2.py
anchor_generator.proto		       keypoint_box_coder.proto
argmax_matcher_pb2.py		       losses_pb2.py
argmax_matcher.proto		       losses.proto
bipartite_matcher_pb2.py	       matcher_pb2.py
bipartite_matcher.proto		       matcher.proto
box_coder_pb2.py		       mean_stddev_box_coder_pb2.py
box_coder.proto			       mean_stddev_box_coder.proto
box_predictor_pb2.py		       model_pb2.py
box_predictor.proto		       model.proto
calibration_pb2.py		       multiscale_anchor_generator_pb2.py
calibration.proto		       multiscale_anchor_generator.proto
center_net_pb2.py		       optimizer_pb2.py
center_net.proto		       optimizer.proto
eval_pb2.py			       pipeline_pb2.py
eval.proto			       pipeline.proto
faster_rcnn_box_coder_pb2.py	       post_processing_pb2.py
faster_rcnn_box_coder.proto	       post_processing.proto
faster_rcnn_pb2.py		       preprocessor_pb2.py
faster_rcnn.proto		       preprocessor.proto
flexible_grid_anchor_generator_p

In [34]:
# 7. 필요한 모듈을 설치
# 필요한 모듈 정보 특정 파일을 가지고 잇음
!cp /content/drive/MyDrive/data/TF2_Object_Detection_API/models/research/object_detection/packages/tf2/setup.py .

In [35]:
# 8. 필요한 package를 설치.
!python -m pip install .

Processing /content/drive/MyDrive/data/TF2_Object_Detection_API/models/research
  Preparing metadata (setup.py) ... done
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1697354 sha256=f4229c2d5833d622300fa4332555bd9539bc511497c9f5a4c39dbee7b2150e16
  Stored in directory: /tmp/pip-ephem-wheel-cache-ptdpzoij/wheels/c9/19/8b/6c047263c467cba381a9a7fee1b3b75a10605fc0304dba9a37
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Can't uninstall 'object-detection'. No files were found to uninstall.


In [17]:
# 9. 나중에 pretrained model을 사용해야 하는데 저장할 공간을 미리하나 만들어 놓음
#  $ROOT_FOLDER"/TF2_Object_Detection_API/pretrained_model
!mkdir "$ROOT_FOLDER"/TF2_Object_Detection_API/pretrained_model

mkdir: cannot create directory ‘/content/drive/MyDrive/data/TF2_Object_Detection_API/pretrained_model’: File exists


In [18]:
# 10. TFRecord Data set을 준비
# 사용하는 데이터셋은 PASCAL VOC Data Set을 이용
# 일반적으로 많은 vision데이터셋들이 우리한테 제공되는데
# imagenet data set, COCO data set, PASCAL VOC data set
# 일단 먼저 해당 이미지셋 데이터를 받아야함
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar

--2023-11-05 13:56:44--  http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1999639040 (1.9G) [application/x-tar]
Saving to: ‘VOCtrainval_11-May-2012.tar.1’

VOCtrainval_11-May- 100%[===================>]   1.86G  36.3MB/s    in 58s     

2023-11-05 13:57:42 (33.0 MB/s) - ‘VOCtrainval_11-May-2012.tar.1’ saved [1999639040/1999639040]



In [19]:
# 11. 이렇게 이미지 tar파일을 받았으면 tar 압축 풀음
!tar -xvf VOCtrainval_11-May-2012.tar

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
VOCdevkit/VOC2012/SegmentationClass/2008_001874.png
VOCdevkit/VOC2012/SegmentationClass/2008_001876.png
VOCdevkit/VOC2012/SegmentationClass/2008_001882.png
VOCdevkit/VOC2012/SegmentationClass/2008_001885.png
VOCdevkit/VOC2012/SegmentationClass/2008_001895.png
VOCdevkit/VOC2012/SegmentationClass/2008_001896.png
VOCdevkit/VOC2012/SegmentationClass/2008_001926.png
VOCdevkit/VOC2012/SegmentationClass/2008_001966.png
VOCdevkit/VOC2012/SegmentationClass/2008_001971.png
VOCdevkit/VOC2012/SegmentationClass/2008_001992.png
VOCdevkit/VOC2012/SegmentationClass/2008_001997.png
VOCdevkit/VOC2012/SegmentationClass/2008_002032.png
VOCdevkit/VOC2012/SegmentationClass/2008_002043.png
VOCdevkit/VOC2012/SegmentationClass/2008_002064.png
VOCdevkit/VOC2012/SegmentationClass/2008_002066.png
VOCdevkit/VOC2012/SegmentationClass/2008_002067.png
VOCdevkit/VOC2012/SegmentationClass/2008_002073.png
VOCdevkit/VOC2012/SegmentationClass/2008_002079.png
VOCdevkit/VOC2012/Segmentati

In [20]:
# 12. TFRecord를 생성해야 하는데 이미 명령어가 있어요
# 그런데 수행을 시킬려면 환경변수를 설정
import os
original_path = os.environ.get('PYTHONPATH') # 기존의 PATH 환경 변수 값을 가져온다.
add_path = ROOT_FOLDER + "/TF2_Object_Detection_API/models/research"
# 새로운 값을 PATH에 추가한다.
os.environ['PYTHONPATH'] = original_path + ':' + add_path

In [21]:
# 13. 설정이 잘 되어있는 지확인
!set

BASH=/bin/bash
BASHOPTS=checkwinsize:cmdhist:complete_fullquote:extquote:force_fignore:globasciiranges:hostcomplete:interactive_comments:progcomp:promptvars:sourcepath
BASH_ALIASES=()
BASH_ARGC=()
BASH_ARGV=()
BASH_CMDS=()
BASH_EXECUTION_STRING=set
BASH_LINENO=()
BASH_SOURCE=()
BASH_VERSINFO=([0]="5" [1]="1" [2]="16" [3]="1" [4]="release" [5]="x86_64-pc-linux-gnu")
BASH_VERSION='5.1.16(1)-release'
CGROUP_MEMORY_EVENTS='/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events'
CLICOLOR=1
CLOUDSDK_CONFIG=/content/.config
CLOUDSDK_PYTHON=python3
COLAB_BACKEND_VERSION=next
COLAB_DEBUG_ADAPTER_MUX_PATH=/usr/local/bin/dap_multiplexer
COLAB_FILE_HANDLER_ADDR=localhost:3453
COLAB_GPU=1
COLAB_JUPYTER_IP=172.28.0.12
COLAB_JUPYTER_TOKEN=
COLAB_JUPYTER_TRANSPORT=ipc
COLAB_KERNEL_MANAGER_PROXY_HOST=172.28.0.12
COLAB_KERNEL_MANAGER_PROXY_PORT=6000
COLAB_LANGUAGE_SERVER_PROXY=/usr/colab/bin/language_service
COLAB_LANGUAGE_SERVER_PROXY_LSP_DIRS=/datalab/web/pyright/typeshed-fallba

In [22]:
# 14.환경변수 설정과 필요한 module 설치, protocol buffer compile과 같은 준비가 다 되었으니 이제
# pascal vod data set을 이용해서
# tfrecord를 만들어보아요
# TFRecord 파일 생성(train)
!python object_detection/dataset_tools/create_pascal_tf_record.py \
    --label_map_path=object_detection/data/pascal_label_map.pbtxt \
    --data_dir=VOCdevkit --year=VOC2012 --set=train \
    --output_path=pascal_train.tfrecord

2023-11-05 14:07:24.743577: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-05 14:07:24.743640: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-05 14:07:24.743680: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-05 14:07:24.752157: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-05 14:07:26.172789: W tensorflow/compiler/

In [23]:
# TFRecord 파일 생성(validation)
!python object_detection/dataset_tools/create_pascal_tf_record.py \
    --label_map_path=object_detection/data/pascal_label_map.pbtxt \
    --data_dir=VOCdevkit --year=VOC2012 --set=val \
    --output_path=pascal_val.tfrecord

2023-11-05 14:08:08.976682: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-05 14:08:08.976750: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-05 14:08:08.976800: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-05 14:08:08.988333: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-05 14:08:11.096968: W tensorflow/compiler/

In [ ]:
#15. 위의 과정이 성공적으로 진행되면 결국
# pascal_val.tfrecordㅡ pascal_train.tfrecord 두개의 파일이
# models/ research 폴더에 만들어지게 됨

# 이제 특정한 폴더구조를 만들어서 object detection model을 학습

In [36]:
# 16. TF2_Object_Detection_API 폴더 밑에 my_od_model이라는 폴더 하나 생성
!mkdir "$ROOT_FOLDER"/TF2_Object_Detection_API/my_od_model

In [37]:
cd "$ROOT_FOLDER"/TF2_Object_Detection_API/my_od_model

/content/drive/MyDrive/data/TF2_Object_Detection_API/my_od_model


In [38]:
#17. my_od_model 폴더 아래 2개의 폴더를 더 만들어야 함
!mkdir "$ROOT_FOLDER"/TF2_Object_Detection_API/my_od_model/data

!mkdir "$ROOT_FOLDER"/TF2_Object_Detection_API/my_od_model/models

In [39]:
# 18. 아까 생성해 놓은 TFRecord 파일 2개를 data
!cp "$ROOT_FOLDER"/TF2_Object_Detection_API/models/research/pascal_train.tfrecord "$ROOT_FOLDER"/TF2_Object_Detection_API/my_od_model/data

!cp "$ROOT_FOLDER"/TF2_Object_Detection_API/models/research/pascal_val.tfrecord "$ROOT_FOLDER"/TF2_Object_Detection_API/my_od_model/data

#label에 대한 데이터파일도 같이 data폴더에 복사

!cp "$ROOT_FOLDER"/TF2_Object_Detection_API/models/research/object_detection/data/pascal_label_map.pbtxt "$ROOT_FOLDER"/TF2_Object_Detection_API/my_od_model/data

In [40]:
# 19. /content/TF2_Object_Detection_API/my_od_model/models 아래에
# pascal_voc_model이라는 폴더를 하나 만들어줌
!mkdir "$ROOT_FOLDER"/TF2_Object_Detection_API/my_od_model/models/pascal_voc_model

In [41]:
#20. 필요한 폴더 구조와 필요한 파일들의 복사는 다 되었음
# 이제 model configuration 을 잡음
# 어떤 방식으로 모델을 학습할건지.. 그내용을 설정
# 설정을 만드는게 쉽지 않음 처음부터 작성하기 힘듦
# 그래서 이미 만들어져 잇는 설정 파일을 수정해서 사용

# 일단 model check point을 하나 받아
# 특정 model에 대한 가중치 파일
# Faster R-CNN ResNet101 V1 800x1333

import os
import six.moves.urllib as urllib
import tarfile

BASE_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'
MODEL_NAME = 'faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8'
MODEL_FILE = MODEL_NAME + '.tar.gz'

DOWNLOAD_URL = BASE_URL + MODEL_FILE
DOWNLOAD_PATH = ROOT_FOLDER + '/TF2_Object_Detection_API/pretrained_model'

if not os.path.isdir(DOWNLOAD_PATH):
    os.mkdir(DOWNLOAD_PATH)

opener = urllib.request.URLopener()
opener.addheader('User-Agent', 'ballentain')
opener.retrieve(DOWNLOAD_URL, filename = DOWNLOAD_PATH + '/' + MODEL_FILE)
print('모델 다운로드 완료...')

tar_file = tarfile.open(DOWNLOAD_PATH + '/' + MODEL_FILE)
tar_file.extractall(DOWNLOAD_PATH)
print('압축 해제 완료...')

모델 다운로드 완료...
압축 해제 완료...


In [42]:
# 21. pretrained model을 다운로드 하고 해당 모델이 가지고 있는 가중치 파일 (check)
!cp -r "$ROOT_FOLDER"/TF2_Object_Detection_API/pretrained_model/faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8/checkpoint "$ROOT_FOLDER"/TF2_Object_Detection_API/my_od_model

In [ ]:
#22. my_model.config파일을 하나 생성
# 생성하는 위치는/content/TF2_Object_Detection_API/my_od_model/models/pascal_voc_model
# Faster R-CNN with Resnet-101 (v1),
# Initialized from Imagenet classification checkpoint
#
# Train on GPU-8
#
# Achieves 36.6 mAP on COCO17 val

# model {
#   faster_rcnn {
#     num_classes: 20
#     image_resizer {
#       keep_aspect_ratio_resizer {
#         min_dimension: 800
#         max_dimension: 1333
#         pad_to_max_dimension: true
#       }
#     }
#     feature_extractor {
#       type: 'faster_rcnn_resnet101_keras'
#     }
#     first_stage_anchor_generator {
#       grid_anchor_generator {
#         scales: [0.25, 0.5, 1.0, 2.0]
#         aspect_ratios: [0.5, 1.0, 2.0]
#         height_stride: 16
#         width_stride: 16
#       }
#     }
#     first_stage_box_predictor_conv_hyperparams {
#       op: CONV
#       regularizer {
#         l2_regularizer {
#           weight: 0.0
#         }
#       }
#       initializer {
#         truncated_normal_initializer {
#           stddev: 0.01
#         }
#       }
#     }
#     first_stage_nms_score_threshold: 0.0
#     first_stage_nms_iou_threshold: 0.7
#     first_stage_max_proposals: 300
#     first_stage_localization_loss_weight: 2.0
#     first_stage_objectness_loss_weight: 1.0
#     initial_crop_size: 14
#     maxpool_kernel_size: 2
#     maxpool_stride: 2
#     second_stage_box_predictor {
#       mask_rcnn_box_predictor {
#         use_dropout: false
#         dropout_keep_probability: 1.0
#         fc_hyperparams {
#           op: FC
#           regularizer {
#             l2_regularizer {
#               weight: 0.0
#             }
#           }
#           initializer {
#             variance_scaling_initializer {
#               factor: 1.0
#               uniform: true
#               mode: FAN_AVG
#             }
#           }
#         }
#       }
#     }
#     second_stage_post_processing {
#       batch_non_max_suppression {
#         score_threshold: 0.0
#         iou_threshold: 0.6
#         max_detections_per_class: 100
#         max_total_detections: 100
#       }
#       score_converter: SOFTMAX
#     }
#     second_stage_localization_loss_weight: 2.0
#     second_stage_classification_loss_weight: 1.0
#   }
# }

# train_config: {
#   # AWS 성능상 batch_size를 4로 설정해도 OOM 오류가 발생.
#   batch_size: 1
#   # AWS 성능상 학습에 걸리는 시간이 1step당 0.6초가 걸림
#   # 200,000 step이면 약 33.3 시간이 걸리게 되니 step의 수를
#   # 조절할 필요가 있음.
#   num_steps: 10000
#   optimizer {
#     momentum_optimizer: {
#       learning_rate: {
#         cosine_decay_learning_rate {
#           learning_rate_base: 0.01
#           total_steps: 10000
#           warmup_learning_rate: 0.0
#           warmup_steps: 5000
#         }
#       }
#       momentum_optimizer_value: 0.9
#     }
#     use_moving_average: false
#   }
#   gradient_clipping_by_norm: 10.0
#   fine_tune_checkpoint_version: V2
#   fine_tune_checkpoint: "/content/TF2_Object_Detection_API/my_od_model/checkpoint/ckpt-0"
#   fine_tune_checkpoint_type: "detection"
#   from_detection_checkpoint: true
#   data_augmentation_options {
#     random_horizontal_flip {
#     }
#   }

#   data_augmentation_options {
#     random_adjust_hue {
#     }
#   }

#   data_augmentation_options {
#     random_adjust_contrast {
#     }
#   }

#   data_augmentation_options {
#     random_adjust_saturation {
#     }
#   }

#   data_augmentation_options {
#      random_square_crop_by_scale {
#       scale_min: 0.6
#       scale_max: 1.3
#     }
#   }
# }

# train_input_reader: {
#   label_map_path: "/content/TF2_Object_Detection_API/my_od_model/data/pascal_label_map.pbtxt"
#   tf_record_input_reader {
#     input_path: "/content/TF2_Object_Detection_API/my_od_model/data/pascal_train.tfrecord"
#   }
# }

# eval_config: {
#   metrics_set: "coco_detection_metrics"
#   use_moving_averages: false
#   batch_size: 1;
# }

# eval_input_reader: {
#   label_map_path: "/content/TF2_Object_Detection_API/my_od_model/data/pascal_label_map.pbtxt"
#   shuffle: false
#   num_epochs: 1
#   tf_record_input_reader {
#     input_path: "/content/TF2_Object_Detection_API/my_od_model/data/pascal_val.tfrecord"
#   }
# }

In [47]:
# w준비가 다 되었으면 이제 명령어를 이용해서 학습을 진행
# 학습을 진행하기 전에 tensorflow버전 2.13.0 으로 변경
# 현재 colab의 tensorlow버전 2.14버전인데 이 상태에서는 학습 오류가 발생
!pip install tensorflow==2.13.0

In [48]:
cd /content/drive/MyDrive/data/TF2_Object_Detection_API/models/research

/content/drive/MyDrive/data/TF2_Object_Detection_API/models/research


In [ ]:
!python object_detection/model_main_tf2.py \
    --pipeline_config_path="$ROOT_FOLDER"/TF2_Object_Detection_API/my_od_model/models/pascal_voc_model/my_model.config \
    --model_dir="$ROOT_FOLDER"/TF2_Object_Detection_API/my_od_model/models/pascal_voc_model \
    --alsologtostderr

2023-11-05 15:05:31.479817: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-11-05 15:05:46.118555: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I1105 15:05:46.119678 132991360401408 mirrored_strategy.py:419] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I1105 15:05:46.157646 132991360401408 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1105 15:05:46.157870 132991360401408 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W1105 15:05:46.376847 132991360401408 dep

In [46]:
import os
import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as ipython_image

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [ ]:
# Utilities

def load_image_into_numpy_array(path):
  # Load an image from file into a numpy array.

  # Args:
  #  path: the file path to the image

  # Returns:
  #  uint8 numpy array with shape (img_height, img_width, 3)
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)


def get_keypoint_tuples(eval_config):
  # Return a tuple list of keypoint edges from the eval config.

  # Args:
  #  eval_config: an eval config containing the keypoint edges

  # Returns:
  #  a list of edge tuples, each in the format (start, end)
    tuple_list = []
    kp_list = eval_config.keypoint_edge
    for edge in kp_list:
        tuple_list.append((edge.start, edge.end))
    return tuple_list

In [ ]:
# Build a detection model and load pre-trained model weights

pipeline_config = ROOT_FOLDER + '/TF2_Object_Detection_API/my_od_model/models/pascal_voc_model/my_model.config'
model_dir = ROOT_FOLDER +'/TF2_Object_Detection_API/my_od_model/models/pascal_voc_model'

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# # Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)

# ckpt-1의 값은 수정.
ckpt.restore(os.path.join(model_dir, 'ckpt-9')).expect_partial()

def get_model_detection_function(model):
  # Get a tf.function for detection.

    @tf.function
    def detect_fn(image):
    # Detect objects in image.
        image, shapes = model.preprocess(image)
        prediction_dict = model.predict(image, shapes)
        detections = model.postprocess(prediction_dict, shapes)

        return detections, prediction_dict, tf.reshape(shapes, [-1])

    return detect_fn

detect_fn = get_model_detection_function(detection_model)

In [ ]:
# Load label map data (for plotting).

label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              = label_map_util.get_label_map_dict(label_map, use_display_name=True)

# Visualization

# image_dir = 'models/research/object_detection/test_images/'
# image_path = os.path.join(image_dir, 'image2.jpg')
image_dir = ROOT_FOLDER  + '/TF2_Object_Detection_API/models/research/VOCdevkit/VOC2012/JPEGImages/'
image_path = os.path.join(image_dir, '2008_008021.jpg')
image_np = load_image_into_numpy_array(image_path)

# Things to try:
# Flip horizontally
# image_np = np.fliplr(image_np).copy()

# Convert image to grayscale
# image_np = np.tile(
#     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)
detections, predictions_dict, shapes = detect_fn(input_tensor)

label_id_offset = 1
image_np_with_detections = image_np.copy()

# # Use keypoints if available in detections
keypoints, keypoint_scores = None, None
if 'detection_keypoints' in detections:
    keypoints = detections['detection_keypoints'][0].numpy()
    keypoint_scores = detections['detection_keypoint_scores'][0].numpy()

# print(detections['detection_boxes'][0].numpy())
# print(detections['detection_classes'][0].numpy())
# print(detections['detection_scores'][0].numpy())

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'][0].numpy(),
      (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
      detections['detection_scores'][0].numpy(),
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      # min_score_thresh를 적절히 설정
      min_score_thresh=.70,
      agnostic_mode=False,
      keypoints=keypoints,
      keypoint_scores=keypoint_scores,
      keypoint_edges=get_keypoint_tuples(configs['eval_config'])
      )

# print(type(image_np_with_detections), image_np_with_detections.shape)
tmp = Image.fromarray(image_np_with_detections)
tmp.save('./test_result.jpg')
ipython_image('./test_result.jpg')
# plt.figure(figsize=(12,16))
# plt.imshow(image_np_with_detections)
# plt.show()